<a href="https://colab.research.google.com/github/royam0820/DL/blob/master/Deploying_Pytorch_and_Building_a_REST_Api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deploying Pytorch and Building a REST Api

## API Definition
We will first define our API endpoints, the request and response types. Our API endpoint will be at /predict which takes HTTP POST requests with a `file` parameter which contains the image. The response will be of JSON response containing the prediction:

In [1]:
{"class_id": "n02124075", "class_name": "Egyptian_cat"}

{'class_id': 'n02124075', 'class_name': 'Egyptian_cat'}

## Dependencies


In [2]:
!pip install Flask==1.0.3 torchvision-0.3.0

     |████████████████████████████████| 92kB 3.4MB/s 
  ERROR: Could not find a version that satisfies the requirement torchvision-0.3.0 (from versions: none)
ERROR: No matching distribution found for torchvision-0.3.0


## Simple Web Server - Flask

In [5]:
%%writefile app.py
from werkzeug.wrappers import Request, Response
from flask import Flask
app = Flask(__name__)


@app.route('/')
def hello():
    return 'Hello World!'

Writing app.py


NOTE:  Save the above snippet in a file called app.py and you can now run a Flask development server by typing:

In [11]:
!FLASK_ENV=development FLASK_APP=app.py flask run

 * Serving Flask app "app.py" (lazy loading)
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 100-769-840
^C


## Inference

In the next sections we will focus on writing the inference code. This will involve two parts, one where we prepare the image so that it can be fed to DenseNet and next, we will write the code to get the actual prediction from the model.



### Preparing the image
DenseNet model requires the image to be of 3 channel RGB image of size 224 x 224. We will also normalise the image tensor with the required mean and standard deviation values. You can read more about it [here](https://pytorch.org/docs/stable/torchvision/models.html).

We will use `transforms` from `torchvision` library and build a transform pipeline, which transforms our images as required. You can read more about transforms [here](https://pytorch.org/docs/stable/torchvision/transforms.html).

In [0]:
import io

import torchvision.transforms as transforms
from PIL import Image

def transform_image(image_bytes):
    my_transforms = transforms.Compose([transforms.Resize(255),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)

NOTE:  The above method takes image data in bytes, applies the series of transforms and returns a tensor. To test the above method, read an image file in bytes mode (first replacing ../_static/img/sample_file.jpeg with the actual path to the file on your computer) and see if you get a tensor back:



In [0]:
with open("../_static/img/sample_file.jpeg", 'rb') as f:
    image_bytes = f.read()
    tensor = transform_image(image_bytes=image_bytes)
    print(tensor)

## Prediction
Now will use a pretrained DenseNet 121 model to predict the image class. We will use one from torchvision library, load the model and get an inference. While we’ll be using a pretrained model in this example, you can use this same approach for your own models. See more about loading your models in this [tutorial](https://pytorch.org/tutorials/beginner/saving_loading_models.html).



In [0]:
from torchvision import models

# Make sure to pass `pretrained` as `True` to use the pretrained weights:
model = models.densenet121(pretrained=True)
# Since we are using our model only for inference, switch to `eval` mode:
model.eval()


def get_prediction(image_bytes):
    tensor = transform_image(image_bytes=image_bytes)
    outputs = model.forward(tensor)
    _, y_hat = outputs.max(1)
    return y_hat

NOTE:  The tensor `y_hat` will contain the index of the predicted class id. However, we need a human readable class name. For that we need a class id to name mapping. Download [this file](https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json) as `imagenet_class_index.json` and remember where you saved it (or, if you are following the exact steps in this tutorial, save it in tutorials/_static). This file contains the mapping of ImageNet class id to ImageNet class name. We will load this JSON file and get the class name of the predicted index.

In [0]:
import json

imagenet_class_index = json.load(open('../_static/imagenet_class_index.json'))

def get_prediction(image_bytes):
    tensor = transform_image(image_bytes=image_bytes)
    outputs = model.forward(tensor)
    _, y_hat = outputs.max(1)
    predicted_idx = str(y_hat.item())
    return imagenet_class_index[predicted_idx]

NOTE:  Before using `imagenet_class_index` dictionary, first we will convert tensor value to a string value, since the keys in the `imagenet_class_index` dictionary are strings. We will test our above method:

In [0]:
with open("../_static/img/sample_file.jpeg", 'rb') as f:
    image_bytes = f.read()
    print(get_prediction(image_bytes=image_bytes))


NOTE:  You should get a response like this:

```
['n02124075', 'Egyptian_cat']

```

The first item in array is ImageNet class id and second item is the human readable name.


*NB:  Did you notice that model variable is not part of `get_prediction` method? Or why is model a global variable? Loading a model can be an expensive operation in terms of memory and compute. If we loaded the model in the `get_prediction` method, then it would get unnecessarily loaded every time the method is called. Since, we are building a web server, there could be thousands of requests per second, we should not waste time redundantly loading the model for every inference. So, we keep the model loaded in memory just once. In production systems, it’s necessary to be efficient about your use of compute to be able to serve requests at scale, so you should generally load your model before serving requests.*

## Integrating the Model to our API Server
In this final part we will add our model to our Flask API server. Since our API server is supposed to take an image file, we will update our predict method to read files from the requests:



In [0]:
from flask import request

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        # we will get the file from the request
        file = request.files['file']
        # convert that to bytes
        img_bytes = file.read()
        class_id, class_name = get_prediction(image_bytes=img_bytes)
        return jsonify({'class_id': class_id, 'class_name': class_name})

NOTE:  The `app.py` file is now complete. Following is the full version; replace the paths with the paths where you saved your files and it should run:

In [0]:
import io
import json

from torchvision import models
import torchvision.transforms as transforms
from PIL import Image
from flask import Flask, jsonify, request

app = Flask(__name__)
imagenet_class_index = json.load(open('<PATH/TO/.json/FILE>/imagenet_class_index.json'))
model = models.densenet121(pretrained=True)
model.eval()


def transform_image(image_bytes):
    my_transforms = transforms.Compose([transforms.Resize(255),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)


def get_prediction(image_bytes):
    tensor = transform_image(image_bytes=image_bytes)
    outputs = model.forward(tensor)
    _, y_hat = outputs.max(1)
    predicted_idx = str(y_hat.item())
    return imagenet_class_index[predicted_idx]


@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        file = request.files['file']
        img_bytes = file.read()
        class_id, class_name = get_prediction(image_bytes=img_bytes)
        return jsonify({'class_id': class_id, 'class_name': class_name})


if __name__ == '__main__':
    app.run()


NOTE:  Let’s test our web server! Run:

In [0]:
!FLASK_ENV=development FLASK_APP=app.py flask run

NOTE:  We can use the requests library to send a POST request to our app:

In [0]:
import requests

resp = requests.post("http://localhost:5000/predict",
                     files={"file": open('<PATH/TO/.jpg/FILE>/cat.jpg','rb')})


NOTE: Printing `resp.json()`` will now show the following:

In [0]:
{"class_id": "n02124075", "class_name": "Egyptian_cat"}

## Next Steps

The server we wrote is quite trivial and and may not do everything you need for your production application. So, here are some things you can do to make it better:

- The endpoint `/predict` assumes that always there will be a image file in the request. This may not hold true for all requests. Our user may send image with a different parameter or send no images at all.

- The user may send non-image type files too. Since we are not handling errors, this will break our server. Adding an explicit error handing path that will throw an exception would allow us to better handle the bad inputs

- Even though the model can recognize a large number of classes of images, it may not be able to recognize all images. Enhance the implementation to handle cases when the model does not recognize anything in the image.

- We run the Flask server in the development mode, which is not suitable for deploying in production. You can check out [this tutorial](https://flask.palletsprojects.com/en/1.1.x/tutorial/deploy/) for deploying a Flask server in production.

- You can also add a UI by creating a page with a form which takes the image and displays the prediction. Check out the [demo](https://pytorch-imagenet.herokuapp.com/) of a similar project and its [source code](https://github.com/avinassh/pytorch-flask-api-heroku).

Total running time of the script: ( 0 minutes 1.179 seconds)



# Running Flask inside a Jupyter Notebook

## The server code

In [9]:
import socket
print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))

from flask import Flask
app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

import threading
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':80}).start() 


172.28.0.2
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:80/ (Press CTRL+C to quit)


## The client code

In [0]:
import requests
r = requests.get("http://172.28.0.2/")
print(r.status_code)
print(r.encoding)
print(r.apparent_encoding)
print(r.text)